In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch-lightning

In [88]:
# import module
import torch 
  
# To get the layers and losses for our model
from torch import nn 
import pytorch_lightning as pl 
  
# To get the activation function for our model
import torch.nn.functional as F 
  
# To get MNIST data and transforms
from torchvision import datasets, transforms
  
# To get the optimizer for our model
from torch.optim import SGD 
  

from torch.utils.data import random_split, DataLoader 

In [90]:

class DataModuleMNIST(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
          
        # Directory to store MNIST Data
        self.download_dir = '/content/drive/MyDrive/258/Assignment 6/Dataset'
          
        # Defining batch size of our data
        self.batch_size = 32
          
        # Defining transforms to be applied on the data
        self.transform = transforms.Compose([transforms.Resize((224,244)),transforms.ToTensor()])

    def collate_fn(data):
      img, bbox = data
      zipped = zip(img, bbox)
      return list(zipped)

    def prepare_data(self):
        print("shreya")
        
    def setup(self, stage=None):
        
        
      data = datasets.ImageFolder(self.download_dir, transform = self.transform)
      self.train_data, self.valid_data, self.test_data = random_split(data, [316, 100, 100])
      self.train_data.dataset.transform = self.transform
      self.valid_data.dataset.transform = self.transform
      self.test_data.dataset.transform = self.transform

    def train_dataloader(self):
        
          # Generating train_dataloader
        return DataLoader(self.train_data, 
                          batch_size = self.batch_size)
  
    def val_dataloader(self):
        
          # Generating val_dataloader
        return DataLoader(self.valid_data,
                          batch_size = self.batch_size)
  
    def test_dataloader(self):
        
        # Generating test_dataloader
        return DataLoader(self.test_data,
                          batch_size = self.batch_size)

In [104]:
class model(pl.LightningModule): 
    def __init__(self): 
        

      super(model, self).__init__()
      #print("inside model")
      self.layer1 = torch.nn.Sequential(
          torch.nn.Conv2d(3,28,kernel_size=5),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2))
      self.layer2 = torch.nn.Sequential(
          torch.nn.Conv2d(28,3,kernel_size=2),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2))

      self.dropout1=torch.nn.Dropout(0.25)
      self.fc1=torch.nn.Linear(9558,5000)
      self.dropout2=torch.nn.Dropout(0.08)
      self.fc2=torch.nn.Linear(5000,3)

    def forward(self, x):
      #print("inside forward")
      x=self.layer1(x)
      x=self.layer2(x)
      x=self.dropout1(x)
      x=torch.relu(self.fc1(x.view(x.size(0), -1)))
      x=F.leaky_relu(self.dropout2(x))
      return F.softmax(self.fc2(x))

    
    def configure_optimizers(self):
      return torch.optim.Adam(self.parameters())

    def training_step(self, train_batch, batch_idx): 
      #print("inside training loop")
      x,labels=train_batch
      #print(x.shape)
      #print(labels.shape)
      #doing a forward pass
      pred=self.forward(x)
      #calculating the loss
      loss = F.nll_loss(pred, labels)     
      #logs
      logs={
          "train_loss": loss}
      output={
          #REQUIRED: It ie required for us to return "loss"
          "loss": loss,
          #optional for logging purposes
          "log": logs
      }
      return output
       
    
    # def validation_step(self, valid_batch, batch_idx): 
        
    #     # Defining validation steps for our model
    #     x, y = valid_batch 
    #     logits = self.forward(x) 
    #     loss = self.loss(logits, y)

In [ ]:
clf = model() 
mnist = DataModuleMNIST() 
trainer = pl.Trainer()
trainer.fit(clf, mnist)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name     | Type       | Params
----------------------------------------
0 | layer1   | Sequential | 2.1 K 
1 | layer2   | Sequential | 339   
2 | dropout1 | Dropout    | 0     
3 | fc1      | Linear     | 47.8 M
4 | dropout2 | Dropout    | 0     
5 | fc2      | Linear     | 15.0 K
----------------------------------------
47.8 M    Trainable params
0         Non-trainable params
47.8 M    Total params
191.250   Total estimated model params size (MB)


shreya


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
